<a href="https://colab.research.google.com/github/NamelessOgya/test-cpu-llm/blob/main/gguf_in_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import userdata

In [ ]:
RESULT_DIR   = '/output'#結果を格納するディレクトリ
MOUNT_GDRIVE = True # google driveをマウントするか
REPEAT  = 2 #同じプロンプトで何回繰り返し出力するか

package_name = "lamma_cpp_gpu"
repo_id = "QuantFactory/Borea-Phi-3.5-mini-Instruct-Jp-GGUF"
filename = "Borea-Phi-3.5-mini-Instruct-Jp.Q6_K.gguf"

In [ ]:
!git clone https://github.com/NamelessOgya/test-cpu-llm.git

Cloning into 'test-cpu-llm'...
remote: Enumerating objects: 143, done.
remote: Counting objects: 100% (143/143), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 143 (delta 53), reused 105 (delta 23), pack-reused 0 (from 0)
Receiving objects: 100% (143/143), 36.23 KiB | 7.25 MiB/s, done.
Resolving deltas: 100% (53/53), done.


In [ ]:
%cd test-cpu-llm

/content/test-cpu-llm


In [ ]:
!nvidia-smi
# nvidia ドライバのバージョンにより、install部分を変更

Sun Dec  8 08:21:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# llmへの入力データを作成(手段はなんでもOK)

In [ ]:
import pandas as pd

In [ ]:
sample_df = pd.DataFrame(
  [
    [1, "ひめっちについて説明してください。"],
    [2, "ひめっちの所属しているユニットを教えてください。"],
    [3, "ひめっちのかわいいところを教えてください。"]
  ],
  columns = ["index", "input"]
)

In [ ]:
sample_df.to_csv("./input/sample.csv", index=False)

# 仮想環境の作成

In [ ]:
!pwd

/content/test-cpu-llm


In [ ]:
!./make_env.sh {package_name}

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

# 実験開始

In [ ]:
!pwd

/content/test-cpu-llm


In [ ]:
# 仮想環境の起動
!source ./env/testvenv/bin/activate \
&& python main.py \
--package_name {package_name}\
--repo_id {repo_id}\
--filename {filename}\
--input_csv_name sample.csv

infer for 0 sample..
infer for 1 sample..
infer for 2 sample..


# develop

In [ ]:
from llama_cpp  import Llama

In [ ]:
sample = '''

現在転職を考えています。
これから記載する経歴を参考にして、私のための職務経歴書を作成してください。

職種: 営業
業種: インターネットサービス
スキル: ビジネスレベル英語
自己PR: 粘り強さ、体力

'''

In [ ]:
llm = Llama.from_pretrained(
    repo_id=repo_id,
    filename=filename,
    verbose=False,
    n_ctx = 4096
)

In [ ]:
sample_format = '''
key; 概要
value: これまで行ってきた職務の概要を記載してください。

key: 業務内容
value: これまで行ってきた業務の内容を箇条書きで記載

key: 成果
value: 業務の結果得られた成果を記載

key: 活かしたスキル
value: 職務遂行の際に活かしたスキルを記載

key: 自己PR
value: 自己PRを記載。各項目に関して項目と具体的内容を記載すること

'''

In [ ]:
input = sample + "出力の際は必ず以下のフォーマットを守ってください。これから列挙するkeyに対して値の要件を満たすようなテキストを出力してください。\n" + sample_format

In [ ]:
input

'\n\n現在転職を考えています。\nこれから記載する経歴を参考にして、私のための職務経歴書を作成してください。\n\n職種: 営業\n業種: インターネットサービス\nスキル: ビジネスレベル英語\n自己PR: 粘り強さ、体力\n\n出力の際は必ず以下のフォーマットを守ってください。これから列挙するkeyに対して値の要件を満たすようなテキストを出力してください。\n\nkey; 概要\nvalue: これまで行ってきた職務の概要を記載してください。\n\nkey: 業務内容\nvalue: これまで行ってきた業務の内容を箇条書きで記載\n\nkey: 成果\nvalue: 業務の結果得られた成果を記載\n\nkey: 活かしたスキル\nvalue: 職務遂行の際に活かしたスキルを記載\n\nkey: 自己PR\nvalue: 自己PRを記載。各項目に関して項目と具体的内容を記載すること\n\n'

In [ ]:
out = llm.create_chat_completion(
    messages=[
        {
            "role": "system",
            "content": "あなたは優秀なキャリアアドバイザーです。転職したいクライアントから相談を受け、クライアントにとって最適な提案を行うことでクライアントを支援します。結果は必ず日本語で出力するルールになっています。出力はjsonフォーマットでお願いします",
        },
        {"role": "user", "content": input},
    ],
    response_format={
        "type": "json_object",
    },
    temperature=0.7,
    max_tokens=3000
)

In [ ]:
out["choices"][0]["message"]["content"]

'{"概要": "インターネットサービス企業で営業として勤務してきた経歴です。粘り強さと体力が私の強みです。","business": ["インターネットサービスの営業として、顧客開拓、営業戦略の立案、提案型営業を行いました。","クライアントのニーズに合ったソリューションを提案し、契約を獲得することを目標に、営業活動を展開していました。","新規顧客獲得、既存顧客の深耕、リードの追客を通して、目標の達成を目指してきました。"],"achievement": ["新規顧客獲得の達成率は30%以上を維持し、既存顧客との関係を深化させることで、リピートビジネスや紹介を獲得することができました。","粘り強く交渉し、困難な案件を成功に導くことができたため、信頼を得ることができました。","結果として、目標の達成はもちろん、顧客との信頼関係を構築することができました。"],"skill": ["ビジネスレベルの英語を使用して、海外の顧客やパートナーとの交渉やコミュニケーションを取ることができました。","粘り強さと体力を活かして、困難な交渉や営業活動を通して成果を得ることができました。"],"自己PR": ["私は粘り強い性格と体力で、困難な案件を成功に導くことができます。粘り強く交渉し、顧客のニーズを理解することで、信頼を得ることができます。インターネットサービス業界での経験を活かして、顧客のニーズに合ったソリューションを提案することができます。"]}'

# gitへのpush

In [ ]:
%cd /content/test-cpu-llm
# !git pull --ff-only
!git status

/content/test-cpu-llm
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   .gitignore
	modified:   output/result_sample.csv

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	env/

no changes added to commit (use "git add" and/or "git commit -a")


In [ ]:
!git add -n .

add '.gitignore'
add 'output/result_sample.csv'
add 'env/.gitkeep'


In [ ]:
!git add --all .

In [ ]:
!git config --global user.email "hakuginnnohana@gmail.com"
!git config --global user.name "NamelessOgya"

In [ ]:
!git commit -m"GPUでの実行に成功"

On branch main
Your branch is ahead of 'origin/main' by 1 commit.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


In [ ]:
!git branch

* main


In [ ]:
push_url_with_pat = "https://" + userdata.get('pat') + "@github.com/"+ userdata.get('git_user_name') + "/test-cpu-llm.git"
!git remote set-url origin {push_url_with_pat}

In [ ]:
# !git remote -v

In [ ]:
!git push origin

Enumerating objects: 10, done.
Counting objects: 100% (10/10), done.
Delta compression using up to 8 threads
Compressing objects: 100% (5/5), done.
Writing objects: 100% (6/6), 1.25 KiB | 1.25 MiB/s, done.
Total 6 (delta 2), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To https://github.com/NamelessOgya/test-cpu-llm.git
   1ca97ee..938e415  main -> main


In [ ]:
!git fetch origin

In [ ]:
!git checkout --theirs sample_nb.ipynb

Updated 1 path from the index


In [ ]:
!git add sample_nb.ipynb

In [ ]:
!git rebase --continue

Successfully rebased and updated refs/heads/main.
